In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Basic imports
import hvplot.xarray
import numpy as np
import xarray as xr
import xdatasets as xd

import xhydro as xh
import xhydro.frequency_analysis as xhfa
from xhydro.extreme_value_analysis.parameterestimation import *
import xclim.indices.stats as stats # not in original example
from scipy.stats import genextreme, gumbel_r, genpareto # not in original example

### Dataset Setup

In [ ]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

ds

In [ ]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {
        "season": ["DJF"],
        "freq": "YS-DEC",
    },  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {},
}

In [ ]:
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

ds_4fa

In [ ]:
# Create a mask beforehand
import random

nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)
dom_end = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[
        {
            "time": slice(
                str(y) + "-04-" + str(dom_start.sel(year=y).item()),
                str(y) + "-06-" + str(dom_end.sel(year=y).item()),
            )
        }
    ] = 1

In [ ]:
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask == 1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            masked,
            op="max",
            timeargs=timeargs_custom,
            missing=missing,
            missing_options=missing_options,
        ),
    ]
)

In [ ]:
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)

# for id in range(len(ds_4fa.coords["id"])):
#     print(ds_4fa.streamflow_max_spring[id,:].values)

print(ds_4fa.streamflow_max_summer[0, :].values)

ds_4fa

In [ ]:
# To speed up the Notebook, we'll only perform the analysis on a subset of variables
# params = xhfa.local.fit(
#     ds_4fa
# )
# print(params.streamflow_max_august[2,0,:].values.tolist())
# params

# extreme_value_analysis.fit()

In [ ]:
from xarray import open_zarr
import glob
import os

path_hist = glob.glob(os.path.join(r"C:\Users\KAMIL PC\Desktop\Life\Stage\Documents\non_stationary_data\eau_precpitable_CanESM5"))
ds_hist = open_zarr(path_hist[0], consolidated=True)
ds_hist

In [ ]:
params_extremes = fit(ds_hist, dist = genpareto, method = "ml")
params_extremes
# params_extremes.precipitable_water[:, 0, 0].plot()

# xclim.stats.fit()

In [ ]:
from lmoments3.distr import gev, gum, gpa
params = stats.fit(ds_hist, dist = gpa, method="ml")
params

In [ ]:
params.precipitable_water[:, 0, 0].plot()

### DEBUG

In [ ]:
# da = xr.DataArray(np.arange(125).reshape(5, 5, 5), dims=("x", "y", "z"))
# da

In [ ]:
# tgt_x = xr.DataArray(np.arange(0, 5), dims="points")
# tgt_y = xr.DataArray(np.arange(0, 5), dims="points")
# da2 = da.isel(x=2, y=2, z = 2)
# da2

In [ ]:
# params_extremes.precipitable_water.isel(y = 2, x = 2, dparams = 1, missing_dims = "raise")
# params_extremes.precipitable_water[2, 2, 1]